In [17]:
%pylab inline
import pylab as pl
import numpy as np
import networkx as nx
import time
import random
import pandas as pd
import statsmodels.formula.api as smf
from copy import deepcopy as dc

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
class Diffuse: #默认网络结构为节点数量为10000，边为60000的单向随机网络
    repetes = 10 #默认多次扩散重复次数
    def __init__(self,p,q,num_runs,G=nx.gnm_random_graph(10000,30000)):
        self.p = p
        self.q = q
        self.G = G.to_directed()
        self.num_runs = num_runs
        
    def decision(self,i): #个体状态决策规则
        influ = len([k for k in self.DG[i].get('prede',[]) if self.DG[k]['state'] == 1])           
        prob = self.p + self.q*influ
        if random.random() <= prob:
            return True
        else:
            return False
        
    def single_diffuse(self): #单次扩散
        self.DG = dc(self.G) #取网络的深层copy，以使原网络不被“污染” 
        for i in self.DG.nodes():
            self.DG[i]['prede'] = self.DG.predecessors(i)
            self.DG[i]['state'] = 0
        non_adopt_set = [i for i in self.DG.nodes() if self.DG[i]['state'] == 0]
        num_of_adopt = []
        j = 1
        while j <= self.num_runs:                                 
            x = 0
            random.shuffle(non_adopt_set)
            for i in non_adopt_set:                   
                if self.decision(i):
                    self.DG[i]['state'] = 1
                    non_adopt_set.remove(i)
                    x = x+1
            num_of_adopt.append(x)
            j = j+1
        return num_of_adopt
    
    def repete_diffuse(self): #多次扩散
        adopt_cont = []
        for i in range(self.repetes):
            num_of_adopt = self.single_diffuse()
            adopt_cont.append(num_of_adopt)
        return adopt_cont

In [12]:
class Random_Grid_Search:
    t_n = 500 #抽样量
    c_n = 50 #保留参数量
    threshold = 1e-6 #循环停止阈值
    orig_points = [] #初始化边界点
    def __init__(self,s): #初始化实例参数
        self.s = np.array(s)  #待拟合曲线
        self.s_len = len(s)
        self.para_range = [[1e-6,0.1],[1e-3,1],[sum(s),4*sum(s)]]  #参数范围
        self.p_range = [[1e-6,0.1],[1e-3,1],[sum(s),4*sum(s)]]  #用于产生边界节点的参数范围 
    
    def gener_orig(self): #递归产生边界点
        if len(self.p_range) == 0:
            return
        else:  
            pa = self.p_range[-1]
            if self.orig_points == []:
                self.orig_points = [[pa[0]],[pa[1]]]  #初始化,排除orig_points为空的情形
            else:
                self.orig_points = [[pa[0]]+x for x in self.orig_points]+[[pa[1]]+x for x in self.orig_points]  #二分裂
            self.p_range.pop()
            return self.gener_orig()
    
    def sample(self,c_range): #抽样参数点
        p_list = []
        for pa in c_range:
            if isinstance(pa[0],float):
                x = (pa[1]-pa[0])*np.random.random(self.t_n) + pa[0]
            else:
                x = np.random.randint(low=pa[0],high=pa[1]+1,size=self.t_n)
            p_list.append(x)

        p_list = np.array(p_list).T
        return p_list.tolist()
    
    def f(self,params): #扩散函数
        diffu_cont = np.zeros(self.s_len)
        t_list = np.arange(1,self.s_len+1)
        a = np.array([1 - np.exp(-(params[0]+params[1])*t) for t in t_list])
        b = np.array([1 + params[1]/params[0]*np.exp(-(params[0]+params[1])*t) for t in t_list])
        diffu_cont = params[2]*a/b

        adopt_cont = np.zeros_like(diffu_cont)
        adopt_cont[0] = diffu_cont[0]
        for t in xrange(1,self.s_len):
            adopt_cont[t] = diffu_cont[t] - diffu_cont[t-1]

        return adopt_cont
    
    def r2(self,params):
        f_act = self.f(params)
        tse = np.sum(np.square(self.s-f_act))
        mean_y = np.mean(self.s)
        ssl = np.sum(np.square(self.s-mean_y))
        R_2 = (ssl-tse)/ssl
        return R_2

    def mse(self,params):  #定义适应度函数（mse）
        a = self.f(params)
        sse = np.sum(np.square(self.s-a))
        return np.sqrt(sse)/self.s_len #均方误
    
    def optima_search(self):
        self.gener_orig()
        c_range = dc(self.para_range)
        samp = self.sample(c_range)
        solution = sorted([self.mse(x)]+x for x in samp+self.orig_points)[:self.c_n]
        u = 1
        while 1:
            params_min = np.min(np.array(solution),0) #最小值
            params_max = np.max(np.array(solution),0) #最大值
            c_range = [[params_min[j+1],params_max[j+1]] for j in range(len(c_range))] #重新定界
            samp = self.sample(c_range)
            solution = sorted([[self.mse(x)]+x for x in samp]+solution)[:self.c_n]
            r = sorted([x[0] for x in solution])
            v = (r[-1]-r[0])/r[0]
            if v < self.threshold:        
                break
            if u > 100:
                print 'Searching ends in 100 runs'
                break
            u+=1
        return solution[0]  #sse,p,q,m

#### pq_range = [(i,j) for i in np.arange(0.0001,0.05,0.0025) for j in np.arange(0.01,0.3,0.005)]  #20*58=1160

In [10]:
k = 0
diff_cont = []
for p,q in pq_range:
    diff = Diffuse(p,q,30)
    temp = diff.repete_diffuse()
    x = np.mean(temp,axis=0)
    x = np.insert(x,0,[p,q])
    diff_cont.append(x)
    k = k+1
    time1 = time.clock()
    if k in [100,400,800,1200]:
        print '========================================'
        print k,'runs:',time.clock()-time1,'s'

to_write = np.array(diff_cont)
np.save("diffusion data set for er (wider range of pq)",to_write)

100 runs: 6.12971327882e-05 s
400 runs: 0.00047099006224 s
800 runs: 0.00043107565034 s


In [ ]:
diff_cont = np.load('diffusion data set for er (wider range of pq).npy')

#### 拟合扩散数据集

In [14]:
coeff_cont = []
for x in diff_cont:
    time1 = time.clock()
    s_full = x[2:]
    len_f = len(s_full)
    max_index = np.argmax(s_full)
    s_act = s_full[:max_index+2]
    RGS = Random_Grid_Search(s_act)
    mse,P,Q,M = RGS.optima_search()
    r2 = RGS.r2([P,Q,M ])
    coeff_cont.append([s_full[0],s_full[1],P,Q,M,r2])
    print x[:2], 'P:%.3f Q:%.3f M:%.3f R2:%.4f'%(P,Q,M,r2)

coeff_cont = np.array(coeff_cont)

[ 0.0001  0.01  ] P:0.003 Q:0.080 M:265.764 R2:0.8466
[ 0.0001  0.015 ] P:0.002 Q:0.099 M:447.898 R2:0.9329
[ 0.0001  0.02  ] P:0.001 Q:0.129 M:865.742 R2:0.9725
[ 0.0001  0.025 ] P:0.001 Q:0.155 M:1367.290 R2:0.9890
[ 0.0001  0.03  ] P:0.000 Q:0.184 M:2502.697 R2:0.9956
[ 0.0001  0.035 ] P:0.000 Q:0.199 M:4812.384 R2:0.9981
[ 0.0001  0.04  ] P:0.000 Q:0.204 M:7259.235 R2:0.9993
[ 0.0001  0.045 ] P:0.001 Q:0.150 M:10324.219 R2:0.9171
[ 0.0001  0.05  ] P:0.000 Q:0.210 M:10012.950 R2:0.9974
[ 0.0001  0.055 ] P:0.000 Q:0.207 M:11063.346 R2:0.9913
[ 0.0001  0.06  ] P:0.000 Q:0.258 M:9792.279 R2:0.9977
[ 0.0001  0.065 ] P:0.001 Q:0.222 M:11697.413 R2:0.9837
[ 0.0001  0.07  ] P:0.001 Q:0.268 M:11551.940 R2:0.9925
[ 0.0001  0.075 ] P:0.000 Q:0.305 M:10465.515 R2:0.9955
[ 0.0001  0.08  ] P:0.000 Q:0.325 M:10167.892 R2:0.9962
[ 0.0001  0.085 ] P:0.001 Q:0.288 M:12350.978 R2:0.9771
[ 0.0001  0.09  ] P:0.000 Q:0.366 M:9647.678 R2:0.9969
[ 0.0001  0.095 ] P:0.000 Q:0.427 M:8654.717 R2:0.9990
[ 0.0

#### 分析$p,q$和$\hat P,\hat Q$之间的关系

In [18]:
to_fit = {}
to_fit['p'] = coeff_cont[:,0]
to_fit['q'] = coeff_cont[:,1]
to_fit['P'] = coeff_cont[:,2]
to_fit['Q'] = coeff_cont[:,3]
to_fit = pd.DataFrame(to_fit)

In [29]:
result_p = smf.ols('p~P+Q-1',data=to_fit).fit()
a = result_p.params
r2 = result_p.rsquared

In [31]:
result_p.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      p   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 4.665e+04
Date:                Mon, 27 Mar 2017   Prob (F-statistic):               0.00
Time:                        18:32:33   Log-Likelihood:                -6169.2
No. Observations:                1160   AIC:                         1.234e+04
Df Residuals:                    1158   BIC:                         1.235e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
P           1.112e+04     72.459    153.487      0.000       1.1e+04  1.13e+04
Q              7.0386      3.944      1.785      0.075        -0.700    14.777
==============================================================================
Omnibus:                      474.043   Durbin-Watson:                   0.446
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9326.850
Skew:                          -1.387   Prob(JB):                         0.00
Kurtosis:                      16.611   Cond. No.                         36.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
result_q = smf.ols('q~P+Q-1',data=to_fit).fit()
b = result_q.params
r2_1 = result_q.rsquared

In [33]:
result_q.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      q   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     8833.
Date:                Mon, 27 Mar 2017   Prob (F-statistic):               0.00
Time:                        18:33:11   Log-Likelihood:                -8037.7
No. Observations:                1160   AIC:                         1.608e+04
Df Residuals:                    1158   BIC:                         1.609e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
P            1.85e+04    362.774     50.987      0.000      1.78e+04  1.92e+04
Q            362.5946     19.746     18.363      0.000       323.853   401.337
==============================================================================
Omnibus:                       57.848   Durbin-Watson:                   0.207
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.528
Skew:                           0.579   Prob(JB):                     5.90e-15
Kurtosis:                       3.121   Cond. No.                         36.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
dict_data = {}
for txt in text_set:
    data = np.load('diffusion data set for %s.npy'%txt)
    num_d = len(data)
    p_cont = data[:,0]
    q_cont = data[:,1]
    print '-------------------------%s--------------------------------'%txt
    print 'number of points:%s'%num_d
    print 'minimal p:%s,maximal p:%s'%(min(p_cont),max(p_cont))
    print 'minimal q:%s,maximal q:%s'%(min(q_cont),max(q_cont))
    dict_data[txt] = data

In [ ]:
fig = pl.figure(figsize=(8,6))
ax = fig.add_subplot(1,1,1)
i = 0
for x in  er_data:
    if i < 600:
        ax.plot(x[2:],'r-',lw=0.1,alpha=0.2)
    else:
        ax.plot(x[2:],'r-',lw=0.1,alpha=0.2)
    
    i = i+1
    ax.set_xlabel('Steps')
    ax.set_ylabel('Number of adopters')